In [19]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
from skfuzzy import cluster

In [3]:
# Global Variables
plt.rcParams['figure.figsize'] = [15, 5]

dfSlap = pd.DataFrame()
dfWrist = pd.DataFrame()
dfHandling = pd.DataFrame()
dfBench = pd.DataFrame()

slap_dtw_dists = []   # 3D array holding (x, y, z) data for each sensor
wrist_dtw_dists = []
handling_dtw_dists = []
bench_dtw_dists = []

acc_ctr = {}          # dictionaries holding DTW distance cluster centers
mag_ctr = {}          # ADDED
gyro_ctr = {}

action_types = ['slap', 'wrist', 'stick_handling', 'bench']  # used in getting test data
act_i = 0
action_num = 1

actions_dtw = {}

predictions = []

In [4]:
# Import csv
def import_csv(fn):
    data = pd.read_csv(fn)
    data.columns = ['time', 'action_type_id', 'action_test_num', 'action_time', 'acc_x', 'acc_y', 'acc_z', 'mag_x', 'mag_y', 'mag_z', 'gyro_x', 'gyro_y', 'gyro_z']
    data.set_index('time')
    return data

In [5]:
# Import training data
def import_training_data():
    global dfSlap, dfWrist, dfHandling, dfBench
    dfSlap = import_csv("training/slap_data.csv")
    dfWrist = import_csv("training/wrist_data.csv")
    dfHandling = import_csv("training/stick_handling_data.csv")
    dfBench = import_csv("training/bench_data.csv")

In [6]:
# Import DTW Distances
def import_dtw_distances():
    global slap_dtw_dists, wrist_dtw_dists, handling_dtw_dists, bench_dtw_dists
    slap_dtw_dists = [pd.read_csv('dtw/slap_accd.csv').values.tolist()]
    slap_dtw_dists.append(pd.read_csv('dtw/slap_magd.csv').values.tolist())
    slap_dtw_dists.append(pd.read_csv('dtw/slap_gyrod.csv').values.tolist())
    wrist_dtw_dists = [pd.read_csv('dtw/wrist_accd.csv').values.tolist()]
    wrist_dtw_dists.append(pd.read_csv('dtw/wrist_magd.csv').values.tolist())
    wrist_dtw_dists.append(pd.read_csv('dtw/wrist_gyrod.csv').values.tolist())
    handling_dtw_dists = [pd.read_csv('dtw/handling_accd.csv').values.tolist()]
    handling_dtw_dists.append(pd.read_csv('dtw/handling_magd.csv').values.tolist())
    handling_dtw_dists.append(pd.read_csv('dtw/handling_gyrod.csv').values.tolist())
    bench_dtw_dists = [pd.read_csv('dtw/bench_accd.csv').values.tolist()]
    bench_dtw_dists.append(pd.read_csv('dtw/bench_magd.csv').values.tolist())
    bench_dtw_dists.append(pd.read_csv('dtw/bench_gyrod.csv').values.tolist())

In [ ]:
# Calculate Dynamic Time Warping distances
def calc_dtw(df, sensor):
    y1 = sensor + '_x'
    y2 = sensor + '_y'
    y3 = sensor + '_z'
    dists = []
    x_dists = []
    y_dists = []
    z_dists = []
    
    for j in range(1,101):
        d1 = df.loc[data['action_test_num'] == j]
        for i in [x for x in range(j,101) if x != j]:
            try:
                d2 = df.loc[data['action_test_num'] == i]
                dx, px = fastdtw(d1[['action_time',y1]], d2[['action_time', y1]], dist=euclidean)
                dy, py = fastdtw(d1[['action_time',y2]], d2[['action_time', y2]], dist=euclidean)
                dz, pz = fastdtw(d1[['action_time',y3]], d2[['action_time', y3]], dist=euclidean)
            except:
                # print(i)
                pass

            x_dists.append(dx)
            y_dists.append(dy)
            z_dists.append(dz)


    dists.append(x_dists)
    dists.append(y_dists)
    dists.append(z_dists)
    return dists

In [7]:
# Find centers of DTW distance clusters (don't care about mag)
def find_cluster_centers():
    global slap_dtw_dists, wrist_dtw_dists, handling_dtw_dists, bench_dtw_dists, acc_ctr, gyro_ctr
    acc_ctr['slap']     = [[np.average(slap_dtw_dists[0][0])], [np.average(slap_dtw_dists[0][1])], [np.average(slap_dtw_dists[0][2])]]
    acc_ctr['wrist']    = [[np.average(wrist_dtw_dists[0][0])], [np.average(wrist_dtw_dists[0][1])], [np.average(wrist_dtw_dists[0][2])]]
    acc_ctr['handling'] = [[np.average(handling_dtw_dists[0][0])], [np.average(handling_dtw_dists[0][1])], [np.average(handling_dtw_dists[0][2])]]
    acc_ctr['bench']    = [[np.average(bench_dtw_dists[0][0])], [np.average(bench_dtw_dists[0][1])], [np.average(bench_dtw_dists[0][2])]]
    # ADDED:
    mag_ctr['slap']     = [[np.average(slap_dtw_dists[1][0])], [np.average(slap_dtw_dists[1][1])], [np.average(slap_dtw_dists[1][2])]]
    mag_ctr['wrist']    = [[np.average(wrist_dtw_dists[1][0])], [np.average(wrist_dtw_dists[1][1])], [np.average(wrist_dtw_dists[1][2])]]
    mag_ctr['handling'] = [[np.average(handling_dtw_dists[1][0])], [np.average(handling_dtw_dists[1][1])], [np.average(handling_dtw_dists[1][2])]]
    mag_ctr['bench']    = [[np.average(bench_dtw_dists[1][0])], [np.average(bench_dtw_dists[1][1])], [np.average(bench_dtw_dists[1][2])]]
    # END ADDED
    gyro_ctr['slap']    = [[np.average(slap_dtw_dists[2][0])], [np.average(slap_dtw_dists[2][1])], [np.average(slap_dtw_dists[2][2])]]
    gyro_ctr['wrist']   = [[np.average(wrist_dtw_dists[2][0])], [np.average(wrist_dtw_dists[2][1])], [np.average(wrist_dtw_dists[2][2])]]
    gyro_ctr['handling']= [[np.average(handling_dtw_dists[2][0])], [np.average(handling_dtw_dists[2][1])], [np.average(handling_dtw_dists[2][2])]]
    gyro_ctr['bench']   = [[np.average(bench_dtw_dists[2][0])], [np.average(bench_dtw_dists[2][1])], [np.average(bench_dtw_dists[2][2])]]

In [8]:
# Get new data from the sensor
# TODO: Will need to change to get NEW data from the sql database
# See: pandas.read_sql_table docs
def get_new_data():
    global action_types, act_i, action_num
    df = pd.read_csv('training/'+action_types[act_i]+'_data.csv')
    df.columns = ['time', 'action_type_id', 'action_test_num', 'action_time', 'acc_x', 'acc_y', 'acc_z', 'mag_x', 'mag_y', 'mag_z', 'gyro_x', 'gyro_y', 'gyro_z']
    df.set_index('time')
    dfAction = df.loc[df['action_test_num'] == action_num]
    return dfAction

In [9]:
# Finds the DTW dist from a single sensor
def sensor_dtw_dists(df1, df2, sensor):
    y1 = sensor + '_x'
    y2 = sensor + '_y'
    y3 = sensor + '_z'
    dists = []
    x_dists = []
    y_dists = []
    z_dists = []
    
    # for i in [x for x in range(1,101)]:
    d2 = df2.loc[df2['action_test_num'] == 1]
    dx, px = fastdtw(df1[['action_time',y1]], d2[['action_time', y1]], dist=euclidean)
    dy, py = fastdtw(df1[['action_time',y2]], d2[['action_time', y2]], dist=euclidean)
    dz, pz = fastdtw(df1[['action_time',y3]], d2[['action_time', y3]], dist=euclidean)

    x_dists.append(dx)
    y_dists.append(dy)
    z_dists.append(dz)
    
    dists.append(x_dists)
    dists.append(y_dists)
    dists.append(z_dists)
    return dists

In [10]:
# Finds the DTW dist from a single group of action training data
def action_dtw_dists(df1, df2):
    sensors_dtw = {}
    sensors_dtw['acc'] = sensor_dtw_dists(df1, df2, 'acc')
    sensors_dtw['mag'] = sensor_dtw_dists(df1, df2, 'mag')
    sensors_dtw['gyro'] = sensor_dtw_dists(df1, df2, 'gyro')
    return sensors_dtw

In [11]:
# Finds the DTW dists for each type of action
def find_dtw_dists(df):
    global actions_dtw
    actions_dtw['slap'] = action_dtw_dists(df, dfSlap)
    actions_dtw['wrist'] = action_dtw_dists(df, dfWrist)
    actions_dtw['handling'] = action_dtw_dists(df, dfHandling)
    actions_dtw['bench'] = action_dtw_dists(df, dfBench)

In [12]:
# Use fuzzy c-means prediction to find cluster membership probabilities
def predict_action():
    # Acc
    new_data = np.hstack((np.array(actions_dtw['slap']['acc']), np.array(actions_dtw['wrist']['acc']), np.array(actions_dtw['handling']['acc']), np.array(actions_dtw['bench']['acc'])))
    ctr = np.vstack((np.array(acc_ctr['slap']).T, np.array(acc_ctr['wrist']).T, np.array(acc_ctr['handling']).T, np.array(acc_ctr['bench']).T))
    ua, uoa, da, _, _, _ = cluster.cmeans_predict(new_data, ctr, 2, error=0.05, maxiter=1000)
    
    # Mag
    new_data = np.hstack((np.array(actions_dtw['slap']['mag']), np.array(actions_dtw['wrist']['mag']), np.array(actions_dtw['handling']['mag']), np.array(actions_dtw['bench']['mag'])))
    ctr = np.vstack((np.array(mag_ctr['slap']).T, np.array(mag_ctr['wrist']).T, np.array(mag_ctr['handling']).T, np.array(mag_ctr['bench']).T))
    um, uom, dm, _, _, _ = cluster.cmeans_predict(new_data, ctr, 2, error=0.05, maxiter=1000)
    
    # Gyro
    new_data = np.hstack((np.array(actions_dtw['slap']['gyro']), np.array(actions_dtw['wrist']['gyro']), np.array(actions_dtw['handling']['gyro']), np.array(actions_dtw['bench']['gyro'])))
    ctr = np.vstack((np.array(gyro_ctr['slap']).T, np.array(gyro_ctr['wrist']).T, np.array(gyro_ctr['handling']).T, np.array(gyro_ctr['bench']).T))
    ug, uog, dg, _, _, _ = cluster.cmeans_predict(new_data, ctr, 2, error=0.05, maxiter=1000)
    
    cma = np.argmax(ua, axis=0) # cluster membership
    cmg = np.argmax(ug, axis=0)
    
    # Predict
    temp = np.add(ua, um)
    u = np.add(temp, ug)
    
    cm = np.argmax(u)
    p = cm % 4

    return p, ua, um, ug, u, cm

In [13]:
# Posts action results to a database
def post_action(p):
    if p != -1:
        print('Shot: ' + action_types[act_i] + ' #' + str(action_num) + '   Predict: ' + action_types[p])
    else:
        print('Shot: ' + action_types[act_i] + ' #' + str(action_num) + '   Predict: ' + 'Unable to predict')
    return

In [14]:
# Main Code
# Setup
import_training_data()
import_dtw_distances()
find_cluster_centers()

# Loop
# for i in range(4):
#     for j in range(1, 101):
#         act_i = i
#         action_num = j
#         dfAction = get_new_data()
#         find_dtw_dists(dfAction)
#         p, ua, um, ug, u, cm = predict_action()
#         post_action(p)

act_i = 1
action_num = 5
dfAction = get_new_data()
find_dtw_dists(dfAction)
p, ua, um, ug, u, cm = predict_action()
post_action(p)

Shot: wrist #5   Predict: wrist
